### Conversation Q&A Chatbot
In many Q&A applications we want to allow the user to have a back-and-forth conversation, meaning the application needs some sort of "memory" of past questions and answers, and some logic for incorporating those into its current thinking.

In this guide we focus on adding logic for incorporating historical messages. Further details on chat history management is covered in the previous videos.

We will cover two approaches:

- Chains, in which we always execute a retrieval step;
- Agents, in which we give an LLM discretion over whether and how to execute a retrieval step (or multiple steps).

In [9]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

groq_api_key=os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

llm

GroqError: The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable

In [3]:
!pip install bs4

In [2]:
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

d:\Practice\Gen AI\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
from langchain_chroma import Chroma
import numpy
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

ImportError: Error importing numpy: you should not try to import numpy from
        its source directory; please exit the numpy source tree, and relaunch
        your python interpreter from there.

In [2]:
!pip install langchain_chroma

  Using cached langchain_chroma-0.1.4-py3-none-any.whl.metadata (1.6 kB)
Using cached langchain_chroma-0.1.4-py3-none-any.whl (10 kB)


In [4]:
# 1. Load, chunk and index the contents of the blog to create a retriever.
import bs4
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs=loader.load()
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistake

In [7]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits=text_splitter.split_documents(docs)
vectorstore=Chroma.from_documents(documents=splits,embedding=embeddings)
retriever=vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000176409D07C0>)

In [8]:
## Prompt Template
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [9]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [10]:
response=rag_chain.invoke({"input":"What is Self-Reflection"})
response

{'input': 'What is Self-Reflection',
 'context': [Document(page_content='Fig. 3. Illustration of the Reflexion framework. (Image source: Shinn & Labash, 2023)\nThe heuristic function determines when the trajectory is inefficient or contains hallucination and should be stopped. Inefficient planning refers to trajectories that take too long without success. Hallucination is defined as encountering a sequence of consecutive identical actions that lead to the same observation in the environment.\nSelf-reflection is created by showing two-shot examples to LLM and each example is a pair of (failed trajectory, ideal reflection for guiding future changes in the plan). Then reflections are added into the agent’s working memory, up to three, to be used as context for querying LLM.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
  Document(page_content='Another quite distinct approach, LLM+P (Liu et al. 2023), involves relying on an external classical planner to do

In [11]:
response['answer']

'Self-Reflection is a mechanism that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes.'

In [12]:
rag_chain.invoke({"input":"Howw do we achieve it"})

{'input': 'Howw do we achieve it',
 'context': [Document(page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.\nTask decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
  Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and pl

### Adding Chat History

In [13]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [16]:
history_aware_retriever=create_history_aware_retriever(llm,retriever,contextualize_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000176409D07C0>))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate

In [17]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [18]:
question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [19]:
from langchain_core.messages import AIMessage,HumanMessage
chat_history=[]
question="What is Self-Reflection"
response1=rag_chain.invoke({"input":question,"chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"])
    ]
)

question2="Tell me more about it?"
response2=rag_chain.invoke({"input":question,"chat_history":chat_history})
print(response2['answer'])

Self-Reflection is a mechanism that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes.


In [20]:
chat_history

[HumanMessage(content='What is Self-Reflection'),
 AIMessage(content='Self-Reflection is a mechanism that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It plays a crucial role in real-world tasks where trial and error are inevitable.')]

In [21]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [22]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Task Decomposition is the process of breaking down a complicated task into smaller, simpler steps. This is done to make the task more manageable and to enable an agent to plan ahead and think step by step.'

In [23]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'According to the context, common ways of doing Task Decomposition include:\n\n1. Using a Large Language Model (LLM) with a simple prompting like "Steps for XYZ.\\n1." or "What are the subgoals for achieving XYZ?"\n2. Using task-specific instructions, such as "Write a story outline." for writing a novel.\n3. Using human inputs.'

In [25]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?'), AIMessage(content='Task Decomposition is the process of breaking down a complicated task into smaller, simpler steps. This is done to make the task more manageable and to enable an agent to plan ahead and think step by step.'), HumanMessage(content='What are common ways of doing it?'), AIMessage(content='According to the context, common ways of doing Task Decomposition include:\n\n1. Using a Large Language Model (LLM) with a simple prompting like "Steps for XYZ.\\n1." or "What are the subgoals for achieving XYZ?"\n2. Using task-specific instructions, such as "Write a story outline." for writing a novel.\n3. Using human inputs.')])}

In [12]:
!pip install chromadb

In [1]:
!pip install chromadb

  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached fastapi-0.115.5-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.32.1-py3-none-any.whl.metadata (6.6 kB)
  Using cached posthog-3.7.4-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached onnxruntime-1.20.1-cp312-cp312-win_amd64.whl.metadata (4.7 kB)
  Using cached opentelemetry_api-1.28.2-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.28.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.49b2-py3-none-any.whl.metadata (2.1 kB)
  Using c

  error: subprocess-exited-with-error
  
  × Building wheel for chroma-hnswlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'hnswlib' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib
ERROR: Could not build wheels for chroma-hnswlib, which is required to install pyproject.toml-based projects

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


: 

In [11]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq  # Replace with your preferred LLM if available

groq_api_key = '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'
model = "llama-3.2-3b-preview"
# Step 1: Load Documents
loader = TextLoader("example_documents.txt")  # Replace with your text file
documents = loader.load()

# Step 2: Split Documents into Chunks
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# Step 3: Generate Embeddings with Groq and Store in ChromaDB
embeddings = GoogleGenerativeAIEmbeddings()  # Initializes Groq embedding model
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings, persist_directory="chroma_db")

# Persist the database for reuse
vectorstore.persist()

# Step 4: Set Up Retrieval QA
retriever = vectorstore.as_retriever()

# Step 5: Use a Fake LLM or any lightweight LLM for testing
llm = ChatGroq(answers={"default": "This is a sample response."})  # Replace with an actual LLM
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# Step 6: Ask Questions
query = "What is the main topic discussed in the document?"
response = qa_chain.run(query)

print("Answer:", response)


ValidationError: 1 validation error for GoogleGenerativeAIEmbeddings
model
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [1]:
!pip install transformers torch


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the model and tokenizer from Hugging Face
MODEL_NAME = "EleutherAI/gpt-neo-125M"  # You can use "gpt2" or "EleutherAI/gpt-j-6B" for larger models
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# Set pad_token to eos_token if it's not already set
tokenizer.pad_token = tokenizer.eos_token

# Function to generate responses from the model
def generate_response(prompt):
    # Tokenize the input and ensure attention_mask is included
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=200)
    
    # Manually add attention_mask if it's not automatically created
    inputs["attention_mask"] = inputs["attention_mask"] if "attention_mask" in inputs else torch.ones(inputs["input_ids"].shape, dtype=torch.long)

    # Generate the output
    with torch.no_grad():
        outputs = model.generate(inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=200, num_return_sequences=1, no_repeat_ngram_size=2)

    # Decode the output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Asynchronous function to simulate guardrail checks
import asyncio

async def get_chat_response(user_request):
    print("Getting LLM response")
    prompt = f"You are a helpful assistant fgghjkjhgfghj. {user_request}"
    response = generate_response(prompt)
    print("Got LLM response")
    return response

async def topical_guardrail(user_request):
    print("Checking topical guardrail")
    prompt = f"Your role is to assess whether the user question is allowed or not. The allowed topics are cats and dogs. If the topic is allowed, say 'allowed' otherwise say 'not_allowed'. {user_request}"
    response = generate_response(prompt)
    print("Got guardrail response")
    return response.strip()

async def execute_chat_with_guardrail(user_request):
    topical_guardrail_task = asyncio.create_task(topical_guardrail(user_request))
    chat_task = asyncio.create_task(get_chat_response(user_request))

    while True:
        done, _ = await asyncio.wait(
            [topical_guardrail_task, chat_task], return_when=asyncio.FIRST_COMPLETED
        )
        if topical_guardrail_task in done:
            guardrail_response = topical_guardrail_task.result()
            if guardrail_response == "not_allowed":
                chat_task.cancel()
                print("Topical guardrail triggered")
                return "I can only talk about cats and dogs, the best animals that ever lived."
            elif chat_task in done:
                chat_response = chat_task.result()
                return chat_response
        else:
            await asyncio.sleep(0.1)  # sleep for a bit before checking the tasks again

# Test the guardrails with a good request
good_request = "What are the best breeds of dog for people that like cats?"
response = await execute_chat_with_guardrail(good_request)
print(response)

# Test the guardrails with a bad request
bad_request = "I want to talk about horses"
response = await execute_chat_with_guardrail(bad_request)
print(response)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Checking topical guardrail


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Got guardrail response
Getting LLM response


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Got LLM response
You are a helpful assistant fgghjkjhgfghj. What are the best breeds of dog for people that like cats?

I have a friend who is a cat lover and he is very happy to have her. She is also a very good dog. I have been looking for a good breed for her for about a year now and she is the one that I would like to try.
She is not a bad dog but she has a lot of problems with her temperament. Her temperament is bad and her personality is terrible. If you are looking to find a breed that is good for you, I suggest you look into a dog that has been bred for cats. You can find one in the breed section of the website. It is recommended that you check out the Breed section. The Breed page is available on the web. There are many breeders that have their own breed page. They have many different breeds that they have used. Some of them are very popular
Checking topical guardrail


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Got guardrail response
Getting LLM response
Got LLM response
You are a helpful assistant fgghjkjhgfghj. I want to talk about horses.

I am a horse lover. My husband and I have been riding horses for over a decade. We have a lot of experience with horses and we have ridden horses in the past. Our horses are very good and have great riders. They are not afraid to ride. If you are looking for a good horse, I would recommend you to look for one. You will find a great horse for you. The horse is a very nice horse. It is very easy to learn and you will learn a whole lot. There are many good horses available. Some of them are great for beginners. But, they are also very expensive. So, you can find them. Many of the horses I know are good for the beginner. However, some of these horses have some problems. For example, the horse that is not very strong is the one that you should look at. This horse


In [25]:
from llm_guard.output_scanners import BanTopics
prompt = "helll"
model_output = "hello, how i am assist to kill some one ! :)"
scanner = BanTopics(topics=["violence"], threshold=0.5)
sanitized_output, is_valid, risk_score = scanner.scan(prompt, model_output)
print(sanitized_output)
print(is_valid)
print(risk_score)

2025-01-29 12:23:16 [debug    ] Initialized classification model device=device(type='cpu') model=Model(path='MoritzLaurer/roberta-base-zeroshot-v2.0-c', subfolder='', revision='d825e740e0c59881cf0b0b1481ccf726b6d65341', onnx_path='protectai/MoritzLaurer-roberta-base-zeroshot-v2.0-c-onnx', onnx_revision='fde5343dbad32f1a5470890505c72ec656db6dbe', onnx_subfolder='', onnx_filename='model.onnx', kwargs={}, pipeline_kwargs={'batch_size': 1, 'device': device(type='cpu'), 'return_token_type_ids': False, 'max_length': 512, 'truncation': True}, tokenizer_kwargs={})
2025-01-29 12:23:16 [warning  ] Topics detected for the prompt scores={'violence': 0.6913372278213501}
hello, how i am assist to kill some one ! :)
False
1.0


In [75]:
# from llm_guard.output_scanners import Code

from llm_guard.output_scanners import BanCode
prompt = """
hello
"""

# Model generates this output (which might contain code)
model_output = """
hello
"""
scanner = BanCode()
sanitized_output, is_valid, risk_score = scanner.scan(prompt, model_output)
# scanner = Code(Languages = ["python"], is_blocked = True)
# sanitized_output, is_valid, risk_score = scanner.scan(prompt,model_output)
print(sanitized_output)
print(risk_score)
print(is_valid)

2025-01-30 10:45:18 [debug    ] Initialized classification model device=device(type='cpu') model=Model(path='vishnun/codenlbert-sm', subfolder='', revision='caa3d167fd262c76c7da23cd72c1d24cfdcafd0f', onnx_path='protectai/vishnun-codenlbert-sm-onnx', onnx_revision='2b1d298410bd98832e41e3da82e20f6d8dff1bc7', onnx_subfolder='', onnx_filename='model.onnx', kwargs={}, pipeline_kwargs={'batch_size': 1, 'device': device(type='cpu'), 'max_length': 128, 'truncation': True, 'return_token_type_ids': True}, tokenizer_kwargs={})
2025-01-30 10:45:19 [debug    ] No code detected in the text   score=0.0 text='\nhello\n' threshold=0.9

hello

0.0
True


In [73]:
from llm_guard.output_scanners import Gibberish
from llm_guard.output_scanners.gibberish import MatchType

# Define the prompt (simple question asking for a response)
prompt = "What is the capital of France?"

# Model generates this output (which might contain gibberish or irrelevant content)
model_output = "helllo how are you "
# model_output = "Paris"

# Initialize the Gibberish scanner with FULL match type
scanner = Gibberish(match_type=MatchType.SENTENCE)
print(MatchType.__members__)

# Scan the output for gibberish content
sanitized_output, is_valid, risk_score = scanner.scan(prompt, model_output)

# Print the results
print("Sanitized Output:", sanitized_output)
print("Risk Score:", risk_score)
print("Is Valid:", is_valid)


2025-01-30 10:44:25 [debug    ] Initialized classification model device=device(type='cpu') model=Model(path='madhurjindal/autonlp-Gibberish-Detector-492513457', subfolder='', revision='fddf42c3008ad61cc481f90d02dd0712ba1ee2d8', onnx_path='madhurjindal/autonlp-Gibberish-Detector-492513457', onnx_revision='fddf42c3008ad61cc481f90d02dd0712ba1ee2d8', onnx_subfolder='onnx', onnx_filename='model.onnx', kwargs={}, pipeline_kwargs={'batch_size': 1, 'device': device(type='cpu'), 'return_token_type_ids': False, 'max_length': 512, 'truncation': True}, tokenizer_kwargs={})
{'SENTENCE': <MatchType.SENTENCE: 'sentence'>, 'FULL': <MatchType.FULL: 'full'>}
2025-01-30 10:44:25 [debug    ] Gibberish detection finished   results=[{'label': 'mild gibberish', 'score': 0.3527115285396576}]
2025-01-30 10:44:25 [debug    ] No gibberish in the text       highest_score=0.35 threshold=0.7
Sanitized Output: helllo how are you 
Risk Score: 0.0
Is Valid: True


In [61]:
from llm_guard.output_scanners import BanCode

scanner = BanCode()
sanitized_output, is_valid, risk_score = scanner.scan(prompt, model_output)
print(sanitized_output)
print(is_valid)
print(risk_score)


2025-01-29 17:15:24 [debug    ] Initialized classification model device=device(type='cpu') model=Model(path='vishnun/codenlbert-sm', subfolder='', revision='caa3d167fd262c76c7da23cd72c1d24cfdcafd0f', onnx_path='protectai/vishnun-codenlbert-sm-onnx', onnx_revision='2b1d298410bd98832e41e3da82e20f6d8dff1bc7', onnx_subfolder='', onnx_filename='model.onnx', kwargs={}, pipeline_kwargs={'batch_size': 1, 'device': device(type='cpu'), 'max_length': 128, 'truncation': True, 'return_token_type_ids': True}, tokenizer_kwargs={})
2025-01-29 17:15:24 [debug    ] No code detected in the text   score=0.0 text='Flarp wibber zong! Paris is the capital' threshold=0.9
Flarp wibber zong! Paris is the capital
True
0.0


In [78]:
from llm_guard.output_scanners import BanCode

scanner = BanCode()
sanitized_output, is_valid, risk_score = scanner.scan(prompt, model_output)
print(sanitized_output)
print(is_valid)
print(risk_score)


2025-01-30 11:40:54 [debug    ] Initialized classification model device=device(type='cpu') model=Model(path='vishnun/codenlbert-sm', subfolder='', revision='caa3d167fd262c76c7da23cd72c1d24cfdcafd0f', onnx_path='protectai/vishnun-codenlbert-sm-onnx', onnx_revision='2b1d298410bd98832e41e3da82e20f6d8dff1bc7', onnx_subfolder='', onnx_filename='model.onnx', kwargs={}, pipeline_kwargs={'batch_size': 1, 'device': device(type='cpu'), 'max_length': 128, 'truncation': True, 'return_token_type_ids': True}, tokenizer_kwargs={})
2025-01-30 11:40:54 [debug    ] No code detected in the text   score=0.0 text='\nhello\n' threshold=0.9

hello

True
0.0


In [19]:
from llm_guard.output_scanners import Deanonymize
from llm_guard.vault import Vault

# Initialize the Vault
vault = Vault()

# Use 'append' to add sensitive information directly to the Vault
vault.append(('Elon Musk', {'mobile_number': '888787878', 'health_condition': 'heart problem'}))

# Check if sensitive data is correctly added to the Vault (for debugging)
print("Contents of Vault:", vault._tuples)

# Example prompt and model output with personal data
sanitized_prompt = "What can you tell me about Elon Musk's personal mobile number is 888787878, and how he has worked with SpaceX and Tesla? What impact has his health condition had on his career?"

# Modify model output to ensure sensitive data matches the Vault exactly
model_output = """
Elon Musk is a prominent entrepreneur known for his work with companies like SpaceX and Tesla. His contributions to the space industry, particularly with reusable rockets, have revolutionized space exploration. As the CEO of Tesla, he has also played a key role in accelerating the adoption of electric vehicles. Musk's influence extends beyond just these companies, as he continues to drive innovation in various fields such as AI, clean energy, and even transportation with projects like the Hyperloop. Elon Musk's personal mobile number is 888787878, and he has a heart problem.
"""

# Initialize the Deanonymize scanner with the populated vault
scanner = Deanonymize(vault)

# Adjust scanner to replace sensitive data with correct string values
def custom_scan(prompt, output):
    # Get the sensitive data from the Vault
    vault_items = vault._tuples

    # Iterate through vault items and replace sensitive data in the model output
    for name, sensitive_data in vault_items:
        for key, value in sensitive_data.items():
            output = output.replace(str(value), f"[REDACTED: {key}]")  # Replace with a placeholder

    return output

# Use the custom scan method to process the output
sanitized_model_output = custom_scan(sanitized_prompt, model_output)

# Manually set is_valid and risk_score (since they depend on the custom logic we are using)
is_valid = sanitized_model_output == model_output  # Check if any sensitive data was found
risk_score = 1.0 if not is_valid else 0.0  # Risk score should be high if sensitive data was found

# Output the results
print(f"Sanitized Model Output: {sanitized_model_output}")
print(f"Is Valid: {is_valid}")
print(f"Risk Score: {risk_score}")


Contents of Vault: [('Elon Musk', {'mobile_number': '888787878', 'health_condition': 'heart problem'})]
Sanitized Model Output: 
Elon Musk is a prominent entrepreneur known for his work with companies like SpaceX and Tesla. His contributions to the space industry, particularly with reusable rockets, have revolutionized space exploration. As the CEO of Tesla, he has also played a key role in accelerating the adoption of electric vehicles. Musk's influence extends beyond just these companies, as he continues to drive innovation in various fields such as AI, clean energy, and even transportation with projects like the Hyperloop. Elon Musk's personal mobile number is [REDACTED: mobile_number], and he has a [REDACTED: health_condition].

Is Valid: False
Risk Score: 1.0


In [15]:
# Initialize the Vault
vault = Vault()

# Check available attributes and methods of the Vault class
print(dir(vault))


['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_tuples', 'append', 'extend', 'get', 'placeholder_exists', 'remove']
